In [1]:
from feature_engine import FeatureEngine
import pandas as pd

In [2]:
test_obj = FeatureEngine()

In [7]:
test_obj.test_set.columns[221:]

Index(['tar_last_10_minutes', 'tar_last_30_minutes', 'tar_last_1_hour',
       'tar_last_3_hours', 'tar_last_6_hours', 'tar_last_12_hours',
       'tbr_last_10_minutes', 'tbr_last_30_minutes', 'tbr_last_1_hour',
       'tbr_last_3_hours',
       ...
       'DKAMostRec_More than 12 months ago', 'DKAMostRec_6-12 months ago',
       'DKANumLast12Mon_0.0', 'DKANumLast12Mon_1.0', 'OthGlucLowerMed_No',
       'OthGlucLowerMed_Yes', 'Weight', 'Height', 'PEAbnormal_No',
       'PEAbnormal_Yes'],
      dtype='object', length=599)

In [92]:
"""
TODO:
Add Morlet Mexican Hat Columns IFF FFT does well
Consider Insulin etc.
"""

''